<a href="https://colab.research.google.com/github/jadepanths/Python-PyTorch-Tutorial/blob/main/Build_The_Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Build The Neural Network
The Neural networks comprise of layer or modules that perform operations on data. The [torch.nn](https://pytorch.org/docs/stable/nn.html) namespace provides all the building blocks you need to build your own neural network. All the modules in PyTorch subclasses the [nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html). A neural network is modile itself that consists of other modules/laters. This nested structure allows for building and managing complex architectures easily.<br/>
<br/>
We will build a neural network to classify images in the FasionMNIST dataset.

In [2]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Get Device for training
Training a model on a hardware accelerator like a GPU is faster than traing on a cpu. Therefore, we should check if cuda is available.

__Note:__ To enable cuda, go to "Runtime" -> "change runtime type" -> change hardware accelerator to "GPU".

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cuda device


# Define the Class
We define our neural network bu subclassing _nn.Module_, and initialize the neural network laters in ```__init__```. Every _nn.Module_ subclass implements the operations on input data in the _forward_ method.

In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

After creating an instance if _NeuralNetwork_, we can move it to the _device_ and print its structure.

In [6]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


Note: **ReLu** is like Riemann sums. You can approximate any continuos functions with lots of little reactangles. ReLu activations can produced lots of little rectangles. ReLu can make complicated shapes and approximate maby complicated domains.

To use the model, we pass it the input data. This executes the model’s _forward_, along with some [background operations](https://github.com/pytorch/pytorch/blob/270111b7b611d174967ed204776985cefca9c144/torch/nn/modules/module.py#L866). Do not call model.forward() directly. <br/>
<br/>
Calling the model on the input returns a 10-dimensional tensor with a raw predicted values for each class. We get the prediction probabilities by passign ti through an instance of the ```nn.Softmax``` module.

In [7]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([4], device='cuda:0')


# Model Layers
Break down the layers in the FashionMNIST model. We will take a sample minibatch of 3 images of size 28x28.

In [8]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


## Flatten
We initialize the [nn.Flatten](https://pytorch.org/docs/stable/generated/torch.nn.Flatten.html) layer to convert each 2D 28*28 image into a contiguous of 784 pixel values (the minibatch dimension (at dim=0) is maintained)

In [9]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


## Linear
The [nn.Linear](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html) is a module that applies a linear transformation on the input using its stored weights and biases.

In [10]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


## ReLU
[nn.ReLu](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html) is used between the linear layers. (There is other activations to introduce non-linearity in your model.

Non-linear activations create the complex mapping between the model's input and outputs. They are apllied after the linear transformation to introduce _nonlinearity_.

In [11]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.4911, -0.4534,  0.6115,  0.0976, -0.1669,  0.2323, -0.6934,  0.5175,
         -0.3112,  0.1438,  0.5929,  0.1103, -0.2898, -0.3941, -0.9390,  0.5804,
          0.0720, -0.6585, -0.2521, -0.0539],
        [ 0.4652,  0.0218,  0.7294, -0.1141,  0.0237, -0.0348, -0.5429,  0.8661,
         -0.1934,  0.0566,  0.4220, -0.1997, -0.3871,  0.1104, -0.9119,  0.5691,
          0.4418, -0.3832, -0.5984, -0.1338],
        [ 0.1519, -0.3420,  0.7074,  0.0182, -0.2008, -0.0081, -0.5623,  0.7078,
         -0.0886, -0.1825,  0.2294,  0.5713, -0.1302, -0.3287, -0.7522,  0.3468,
          0.1062, -0.5794, -0.0224, -0.0202]], grad_fn=<AddmmBackward>)


After ReLU: tensor([[0.4911, 0.0000, 0.6115, 0.0976, 0.0000, 0.2323, 0.0000, 0.5175, 0.0000,
         0.1438, 0.5929, 0.1103, 0.0000, 0.0000, 0.0000, 0.5804, 0.0720, 0.0000,
         0.0000, 0.0000],
        [0.4652, 0.0218, 0.7294, 0.0000, 0.0237, 0.0000, 0.0000, 0.8661, 0.0000,
         0.0566, 0.4220, 0.0000, 0.0000, 0.1104, 0.000

## Sequential
[nn.Sequential](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html) is an ordered container of modules. This data is passed through all the modules in the same order as defined. You can use sequential containers to put together a quicl network like *seq_modules*

In [12]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

## Softmax
The last linear layer of the neural network returns *logits* - raw values in [infty, infty] - which are passed to the [nn.Softmax](https://pytorch.org/docs/stable/generated/torch.nn.Softmax.html) module. The logits are scaled to value [0, 1] representing the model's predicted probabilities for each class. ```dim``` parameter indicates the dimension along which the values must sum to 1.

In [16]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)
print(pred_probab)

tensor([[0.0967, 0.0908, 0.1688, 0.0947, 0.0887, 0.0989, 0.0850, 0.0854, 0.0745,
         0.1165],
        [0.1063, 0.0874, 0.1402, 0.0855, 0.0968, 0.0981, 0.0856, 0.0956, 0.0931,
         0.1115],
        [0.0880, 0.0937, 0.1490, 0.0981, 0.0933, 0.1009, 0.0861, 0.0969, 0.0801,
         0.1140]], grad_fn=<SoftmaxBackward>)


## Model Parameters
Many layers inside a neural network are parameterized, i.e. have associated weights and biases that are optimized during training. Subclassing ```nn.Module``` automatically tracks all fields defined inside your model object, and makes all parameters accessible using your model’s ```parameters()``` or ```named_parameters()``` methods.

In this example, we iterate over each parameter, and print its size and a preview of its values.

In [17]:
print("Model structure: ", model, "\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure:  NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
) 


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 8.4528e-03, -2.6379e-02,  4.6932e-03,  ...,  5.4866e-05,
          9.7483e-03,  1.1605e-02],
        [-2.9542e-02, -2.6049e-03,  2.8827e-02,  ...,  1.4241e-03,
          3.4997e-02, -1.0284e-02]], device='cuda:0', grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([0.0343, 0.0138], device='cuda:0', grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0050, -0.0251, -0.0393,  ...,  0.0376,  0.0372,  0.0147],
        [-0.0248,  0.0293, -0.019